In [1]:
experimentsPath = "/raid/elhamod/Fish/experiments/" #"/raid/elhamod/CIFAR_HGNN/experiments/" #"/raid/elhamod/Fish/experiments/"
dataPath = "/raid/elhamod/Fish/" # "/raid/elhamod/" #"/raid/elhamod/Fish/"
experimentName = "Fish_s2_experiments" #"Fish_tripletloss_alpha_lr_experiments"
device = 0
detailed_reporting = False

In [2]:
import matplotlib.pyplot as plt
import torch
import sys
import os
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
from tqdm.auto import trange
import wandb

import warnings
# warnings.filterwarnings("ignore")

try:
    import wandb
except:
    print('wandb not found')

from myhelpers import config_plots, TrialStatistics
from myhelpers.try_warning import try_running
from HGNN.train import CNN, dataLoader
from myhelpers import cifar_dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
config_plots.global_settings()

experimetnsFileName = "experiments.csv"
WANDB_message="wandb not working"


# For logging to server
try_running(lambda : wandb.login(), WANDB_message)

experimentPathAndName = os.path.join(experimentsPath, experimentName)
# set cuda
if device is not None:
    print("using cuda", device)
    torch.cuda.set_device(device)

else:
    print("using cpu")

# get experiment params
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

# init experiments file
experimentsFileNameAndPath = os.path.join(experimentsPath, experimetnsFileName)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  
experiment_index = 0

# Loop through experiments
# with progressbar.ProgressBar(max_value=number_of_experiments) as bar:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for experiment_params in config_parser.getExperiments():
        print(experiment_params)
        experimentHash =TrialStatistics.getTrialName(experiment_params)

        # load images 
        if experiment_params['image_path'] == 'cifar-100-python':
            datasetManager = cifar_dataLoader.datasetManager(experimentPathAndName, dataPath)
        else:
            datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
        datasetManager.updateParams(config_parser.fixPaths(experiment_params))
        train_loader, validation_loader, test_loader = datasetManager.getLoaders()
        architecture = {
            "fine": len(train_loader.dataset.csv_processor.getFineList()),
            "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
        }

        # Loop through n trials
        for i in trange(experiment_params["numOfTrials"], desc="trial"):
            modelName = getModelName(experiment_params, i)
            trialName = os.path.join(experimentPathAndName, modelName)
            trialHash = TrialStatistics.getTrialName(experiment_params, i)

            row_information = {
                'experimentName': experimentName,
                'modelName': modelName,
                'datasetName': getDatasetName(config_parser.fixPaths(experiment_params)),
                'experimentHash': experimentHash,
                'trialHash': trialHash
            }
            row_information = {**row_information, **experiment_params} 
            print(row_information)

            run = try_running(lambda : wandb.init(project='HGNN', group=experimentName+"-"+experimentHash, name=trialHash, config=row_information), WANDB_message) #, reinit=True

            # Train/Load model
            model = CNN.create_model(architecture, experiment_params, device=device)
            
#             from torchsummary import summary

#             summary(model, (3, 224, 224))

#             try_running(lambda : wandb.watch(model, log="all"), WANDB_message)

            if os.path.exists(CNN.getModelFile(trialName)):
                print("Model {0} found!".format(trialName))
            else:
                initModelPath = CNN.getInitModelFile(experimentPathAndName)
                if os.path.exists(initModelPath):
                    model.load_state_dict(torch.load(initModelPath))
                    print("Init Model {0} found!".format(initModelPath))
                CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName, test_loader, device=device, detailed_reporting=detailed_reporting)

            # Add to experiments file
            if os.path.exists(experimentsFileNameAndPath):
                experiments_df = pd.read_csv(experimentsFileNameAndPath)
            else:
                experiments_df = pd.DataFrame()

            record_exists = not (experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName]).empty if not experiments_df.empty else False
            if record_exists:
                experiments_df.drop(experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName].index, inplace = True) 

            experiments_df = experiments_df.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
            experiments_df.to_csv(experimentsFileNameAndPath, header=True, index=False)

            try_running(lambda : run.finish(), WANDB_message)

        bar.update()

        experiment_index = experiment_index + 1



# if __name__ == "__main__":
#     torch.multiprocessing.set_start_method('spawn')
    
#     import argparse

#     parser = argparse.ArgumentParser()
#     parser.add_argument('--cuda', required=True, type=int)
#     parser.add_argument('--experiments', required=True)
#     parser.add_argument('--data', required=True)
#     parser.add_argument('--name', required=True)
#     parser.add_argument('--detailed', required=False, action='store_true')
#     args = parser.parse_args()
#     main(experimentName=args.name, experimentsPath=args.experiments, dataPath=args.data, device=args.cuda, detailed_reporting=args.detailed)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mndhamod (use `wandb login --relogin` to force relogin)
experiment:   0%|          | 0/10 [00:00<?, ?it/s]

using cuda 0
{'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.0001, 'numOfTrials': 1, 'fc_layers': 1, 'pretrained': True, 'epochs': 80, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.01, 'scheduler_gamma': 0.5, 'scheduler_patience': 10, 'modelType': 'BB', 'lambda': 0, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'tripletEnabled': False, 'regularTripletLoss': False, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 2, 'phylogeny_loss': False, 'displayName': 'Fish-s2-BBlonger', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}
Creating datasets...
{'Alosa chrysochloris': 482298, 'Carassius auratus': 1005907, 'Cyprinus carpio': 429083, 'Esox americanus': 496115, 'Gambusia affinis': 617445, 'Lepisosteus osseus': 519445, 'Lepisosteus platostomus': 731608, 'Lepomis auritus': 100

{'experimentName': 'Fish_s2_experiments', 'modelName': 'models/9597809d2f803c57e02d64f9a42d23d1822ccc6d0ad5ca00e6e4068f', 'datasetName': 'datasplits/7c7513bdfb4e5577fef1c3ec0fa6452d30d87dbc95f258a7c86dd76b', 'experimentHash': '0e17525006238fdd6bd48f93259409cda8382b3cd2ee4de330ad17d3', 'trialHash': '9597809d2f803c57e02d64f9a42d23d1822ccc6d0ad5ca00e6e4068f', 'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.0001, 'numOfTrials': 1, 'fc_layers': 1, 'pretrained': True, 'epochs': 80, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.01, 'scheduler_gamma': 0.5, 'scheduler_patience': 10, 'modelType': 'BB', 'lambda': 0, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'tripletEnabled': False, 'regularTripletLoss': False, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 2, 'phylogeny_loss': False, 'd

wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Model /raid/elhamod/Fish/experiments/Fish_s2_experiments/models/9597809d2f803c57e02d64f9a42d23d1822ccc6d0ad5ca00e6e4068f found!


...

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
Training started...




iteration:   0%|          | 0/40 [00:34<?, ?it/s, min_val_loss=inf, train=0.00251, val=0.00179, val_loss=3.64]

iteration:   2%|▎         | 1/40 [00:34<22:36, 34.77s/it, min_val_loss=inf, train=0.00251, val=0.00179, val_loss=3.64]

iteration:   2%|▎         | 1/40 [01:30<22:36, 34.77s/it, min_val_loss=inf, train=0.283, val=0.239, val_loss=3.56]    

iteration:   5%|▌         | 2/40 [01:30<26:00, 41.05s/it, min_val_loss=inf, train=0.283, val=0.239, val_loss=3.56]

iteration:   5%|▌         | 2/40 [02:28<26:00, 41.05s/it, min_val_loss=4.18, train=0.538, val=0.438, val_loss=3.42]

iteration:   8%|▊         | 3/40 [02:28<28:32, 46.27s/it, min_val_loss=4.18, train=0.538, val=0.438, val_loss=3.42]

iteration:   8%|▊         | 3/40 [03:26<28:32, 46.27s/it, min_val_loss=2.29, train=0.831, val=0.675, val_loss=3.36]

iteration:  10%|█         | 4/40 [03:26<29:43, 49.54s/it, min_val_loss=2.29, train=0.831, val=0.675, val_loss=3.36]

iteration:  10%|█         | 4/40 [04:24<29:43, 49.54s/it, min_

epoch,39
learning rate,0.001
validation_fine_f1,0.88384
training_fine_f1,1.0
training_fine_acc,1.0
test_fine_f1,0.89605
test_fine_acc,0.89737
validation_loss,2.85822
_runtime,2352
_timestamp,1622055637
_step,780


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_fine_f1,▁▃▄▆▇▇▇▇▇▇▆▇▇▆▅▇▆▇▇▇▇███████████████████
training_fine_f1,▁▃▅▇▇████████▇▇▇▇███████████████████████
training_fine_acc,▁▃▅▇▇████████▇▇█▇███████████████████████
test_fine_f1,▁▃▄▆▆▆▇▇▇▇▇▇▇▅▅▆▆▇▇▇▇███████████████████
test_fine_acc,▁▃▅▆▆▆▇▇▇▇▇▇▇▅▅▆▆▇▇▇▇███████████████████
validation_loss,█▇▆▅▅▄▄▃▄▃▄▃▂▄▅▃▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


experiment: 100%|██████████| 10/10 [2:04:50<00:00, 749.08s/it] 